# 4. MLflow

## 4.1 MLflow 개념 잡기

### 4.1.1 MLflow가 없던 시절

- 사람들이 각자 자신의 코드를 Jupyter Notebook에서 작성
- 머신러닝 모델 학습시 사용한 Parameter, Metric을 따로 기록

- 개인 컴퓨터, 연구실 서버를 사용하다가 메모리 초과로 Memory Exceed 오류 발생
- 학습하며 생긴 Weight File을 저장해 다른 동료들에게 공유
- Weight File 이름으로 Model Versioning을 하거나 아예 Versioning을 하지 않음

<br>

### 4.1.2 MLflow가 해결하려고 했던 Pain Point

1. 실험을 추적하기 어렵다.
2. 코드를 재현하기 어렵다.
3. 모델을 패키징하고 배포하는 방법이 어렵다.
4. 모델을 관리하기 위한 중앙 저장소가 없다.

<br>

### 4.1.3 MLflow란?

- 머신러닝 실험, 배포를 쉽게 관리할 수 있는 오픈소스
- 관련 오픈소스 중 제일 빠르게 성장 중

<br>

### 4.1.4 MLflow - 맛보기

CLI, GUI(웹 인터페이스) 지원

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
<img src='https://drive.google.com/uc?id=1zNGTD1J9mYvnujF1mqYNP2PUSXqdxeGX' width=800/>

<br>

Code 예시 - `main.py`

```python
import numpy as np
from sklearn.linear_model import LinearRegression
import mlflow

def main():
    # enable autologging
    mlflow.autolog(log_input_examples=True)
    
    # prepare training data
    X = np.array([[1, 1], [1, 2], [2, 2], [2, 3]])
    y = np.dot(X, np.array([1, 2])) + 3
    
    # train a model
    model = LinearRegression()
    with mlflow.start_run() as run:
        model.fit(X, y)
        print("Logged data and model in run {}".format(run.info.run_id))

if __name__ == "__main__":
    main()
```

<br>

### 4.1.5 MLflow의 핵심 기능

#### 4.1.5.1 Experiment Management & Tracking

- 머신러닝 관련 **"실험"들을 관리**하고, 각 실험의 내용들을 **기록**할 수 있음
  - 예를 들어, 여러 사람이 **하나의 MLflow 서버 위에서 각자 자기 실험을 만들고 공유**할 수 있음
- 실험을 정의하고, 실험을 실행할 수 있음. 이 실행은 머신러닝 훈련 코드를 실행한 기록
  - 각 실행에 사용한 소스 코드, 하이퍼 파라미터, Metric, 부산물(모델 Artifact, Chart Image) 등을 저장

<br>

#### 4.1.5.2 Model Registry

- MLflow로 실행한 머신러닝 모델을 Model Registry(모델 저장소)에 등록할 수 있음
- 모델 저장소에 모델이 저장될 때마다 해당 모델에 버전이 자동으로 올라감(Version 1 -> 2 -> 3..)
- Model Registry에 등록된 모델은 다른 사람들에게 쉽게 공유 가능하고, 쉽게 활용할 수 있음
- Github, Docker와 관련

<br>

#### 4.1.5.3 Model Serving

- Model Registry에 등록한 모델을 REST API 형태의 서버로 Serving 할 수 있음
- Input = Model의 Input
- Output = Model의 Output
- 직접 Docker Image 만들지 않아도 생성할 수 있음

<br>

### 4.1.6 MLflow Component

1. MLflow Tracking
2. MLflow Project
3. MLflow Model
4. MLflow Registry

<br>

#### 4.1.6.1 MLflow Tracking

- 머신러닝 코드 실행, 로깅을 위한 API, UI
- MLflow Tracking을 사용해 결과를 Local, Server에 기록해 여러 실행과 비교할 수 있음
- 팀에선 다른 사용자의 결과와 비교하며 협업할 수 있음

<br>

#### 4.1.6.2 MLflow Project

- 머신러닝 프로젝트 코드를 패키징하기 위한 표준
- Project
  - 간단하겐 소스 코드가 저장된 폴더
  - Git Repo
  - 의존성과 어떻게 실행해야 하는지 저장
- MLflow Tracking API를 사용하면 MLflow는 프로젝트 버전을 모든 파라미터와 자동으로 로깅

<br>

#### 4.1.6.3 MLflow Model

- 모델은 모델 파일과 코드로 저장
- 다양한 플랫폼에 배포할 수 있는 여러 도구 제공
- MLflow Tracking API를 사용하면 MLflow는 자동으로 해당 프로젝트에 대한 내용을 사용함

<br>

#### 4.1.6.4 MLflow Registry

- MLflow Model의 전체 Lifecycle에서 사용할 수 있는 중앙 모델 저장소

<br>

## 4.2 MLflow Tracking

### 4.2.1 MLflow 설치

```bash
pip install mlflow
```

<br>

### 4.2.2 MLflow Tracking - Experiment

#### 4.2.2.1 Experiment (실험)

- MLflow에서 제일 먼저 Experiment를 생성
- 하나의 Experiment는 진행하고 있는 머신러닝 프로젝트 단위로 구성
  - ex) "개/고양이 이미지 분류 실험", "택시 수요량 예측 분류 실험"

- 정해진 Metric으로 모델을 평가
  - ex) RMSE, MSE, MAE, Accuracy

- 하나의 Experiment는 여러 Run(실행)을 가짐

<br>

#### 4.2.2.2 Experiment 생성

```bash
mlflow experiments create --experiment-name my-first-experiment
```

- `ls -al`을 사용해 폴더를 확인하면 `mlruns`라는 폴더가 생김
  - `mlruns`는 실험(experiment)에 대한 실행(run)에 대한 정보를 저장하고 있다.

<br>

#### 4.2.2.3 Experiment 리스트 확인

```bash
mlflow experiments list
```

<br>

### 4.2.3 MLflow 머신러닝 코드 작성

#### 4.2.3.1 모델에 필요한 라이브러리 설치

```bash
pip install numpy sklearn
```

<br>

#### 4.2.3.2 폴더 생성 후, 머신러닝 코드 생성

```bash
mkdir logistic_regression
vi logistic_regression/train.py
```

`train.py`

```python
import numpy as np
from sklearn.linear_model import LogisticRegression

import mlflow
import mlflow.sklearn

if __name__ == "__main__":
    X = np.array([-2, -1, 0, 1, 2, 1]).reshape(-1, 1)
    y = np.array([0, 0, 1, 1, 1, 0])
    
    penalty = "elasticnet"
    l1_ratio = 0.1
    lr = LogisticRegression(penalty=penalty, l1_ratio=l1_ratio)
    
    lr.fit(X, y)
    
    score = lr.score(X, y)
    print("Score: %s" % score)
    
    mlflow.log_param("penalty", penalty)
    mlflow.log_param("l1_ratio", 0.1)
    mlflow.log_metric("score", score)
    mlflow.sklearn.log_model(lr, "model")
```

<br>

### 4.2.4 MLflow Project

#### 4.2.4.1 프로젝트 (MLProject)

- MLflow를 사용한 코드의 프로젝트 메타 정보 저장
- 프로젝트를 어떤 환경에서 어떻게 실행시킬지 정의
- 패키지 모듈의 상단에 위치

<br>

#### 4.2.4.2 `MLProject` 생성

```bash
vi logistic_regression/MLProject
```

```
name: tutorial

entry_points:
  main:
    command: "python train.py"
```

<br>

### 4.2.5 MLflow Tracking - Run

#### 4.2.5.1 Run (실행)

- 하나의 Run은 코드를 1번 실행한 것을 의미
- 보통 Run은 모델 학습 코드를 실행
- 즉, 한번의 코드 실행 = 하나의 Run 생성
- Run을 하면 여러가지 내용이 기록됨

<br>

#### 4.2.5.2 Run에서 로깅하는 것들


- Source
  - 실행한 Project의 이름
- Version
  - 실행 Hash
- Start & end time
- Parameters
  - 모델 파라미터
- Metrics
  - 모델의 평가 지표
  - Metric을 시각화할 수 있음
- Tags
  - 관련된 Tag
- Artifacts
  - 실행 과정에서 생기는 다양한 파일들
  - ex) 이미지, 모델 Pickle 등

<br>

#### 4.2.5.3 Run으로 실행

```bash
mlflow run logistic_regression --experiment-name my-first-experiment --no-conda
```

<br>

### 4.2.6 MLflow Tracking - UI

#### 4.2.6.1 UI 실행

```bash
mlflow ui
```

<br>

#### 4.2.6.2 MLflow UI 접속

- `localhost:5000`로 MLflow UI 접속

<br>

### 4.2.7 MLflow Tracking - Experiment / Run 관계

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
<img src='https://drive.google.com/uc?id=1zNrXcGfuI-hT5mKvXJUEGd8Y3b9xBt6J' width=600/>

<br>

### 4.2.8 MLflow autolog

- 파라미터를 매번 명시해야 할까?

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
<img src='https://drive.google.com/uc?id=1zODidnEkx2qjzwJmrzc1k0oZ2qakho0f' width=800/>

- autolog 적용 후 run으로 실행

```bash
mlflow run logistic_regression_with_autolog --experiment-name my-first-experiment --no-conda
```

<br>

- mlflow ui에서 모니터링

<br>

주의사항

- autolog는 모든 프레임워크에서 사용 가능한 것은 아님
- MLflow에서 지원해주는 프레임워크들이 존재
  - ex) `pytorch.nn.Module`은 지원하지 않음 (반면 Pytorch Lightning은 지원)
  - 자세한 내용은 https://mlflow.org/docs/latest/tracking.html#id2 에서 확인

- 현재 autolog 지원 중인 라이브러리

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
<img src='https://drive.google.com/uc?id=1zXG_a2G0jRdCN2vudhSwlHeMdnIuibMU' width=400/>

<br>

### 4.2.9 MLflow Parameter

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
<img src='https://drive.google.com/uc?id=1zPp5_pPYf600QUAZ69OX-6d43cKHM1B1' width=800/>

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
<img src='https://drive.google.com/uc?id=1zTHPR4-phtTYL0N0sAVNgtEXbVzaAvV1' width=800/>

```bash
mlflow run logistic_regression_with_autolog_and_params \
-P solver="saga" \
-P penalty="elasticnet" \
-P l1_ratio=0.01 \
--experiment-name my-first-experiment --no-conda
```

<br>

### 4.2.10 MLflow Hyper Parameter Tuning

- autolog와 하이퍼 파라미터 튜닝도 같이 할 수 있음

```python
from sklearn import svm, datasets
from sklearn.model_selection import GridSearchCV

import mlflow

def main():
    mlflow.sklearn.autolog()
    
    iris = datasets.load_iris()
    parameters = {"kernel": ("linear", "rbf"), "C": [1, 10]}
    svc = svm.SVC()
    clf = GridSearchCV(svc, parameters)
    
    with mlflow.start_run() as run:
        clf.fit(iris.data, iris.target)
        
if __name__ == "__main__":
    main()
```

<br>

```bash
mlflow run svc_with_hyperparams_tuning --experiment-name my-first-experiment --no-conda
```

<br>

## 4.3 MLflow 서버로 배포하기

- 개인 프로젝트를 할 때나 캐글같은 대회에 참여할 때 mlflow를 사용하기 위해 docker image로 만들어 놓으면 좋을 것이다.
- mlflow 서버로 배포하여 협업할 수 있다.

<br>

### 4.3.1 MLflow Architecture

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
<img src='https://drive.google.com/uc?id=1zUDFiVVtQ4HCNI1Pzr5nWIj0N_wGxvhG' width=800/>

1. 파이썬 코드 (with MLflow package)
2. Tracking Server
3. Artifact Store

- 트리 확인

```bash
tree mlruns
```

<br>

#### 4.3.1.1 파이썬 코드 (with MLflow package)

- 모델을 만들고 학습하는 코드
- `mlflow run`으로 실행

```python
from sklearn import svm, datasets
from sklearn.model_selection import GridSearchCV

import mlflow

def main():
    mlflow.sklearn.autolog()
    
    iris = datasets.load_iris()
    parameters = {"kernel": ("linear", "rbf"), "C": [1, 10]}
    svc = svm.SVC()
    clf = GridSearchCV(svc, parameters)
    
    with mlflow.start_run() as run:
        clf.fit(iris.data, iris.target)
        
if __name__ == "__main__":
    main()
```

<br>

#### 4.3.1.2 Tacking Server

- 파이썬 코드가 실행되는 동안 Parameter, Metric, Model 등 메타 정보 저장
- 파일 혹은 DB에 저장

<br>

#### 4.3.1.3 Artifact Store

- 파이썬 코드가 실행되는 동안 생기는 Model File, Image 등의 artifact를 저장
- 파일 혹은 스토리지에 저장

<br>

### 4.3.2 Tracking Server와 외부 스토리지 사용하기

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
<img src='https://drive.google.com/uc?id=1zUI685D3iMwdV2gK5_R3fuYOonbIgigA' width=800/>

<br>

- `mlflow server` 명령어로 Backend Store URI 지정

```bash
mlflow server --backend-store-uri sqlite:///mlflow.db --default-artifact-root $(pwd)/artifacts
```

<br>

- 환경변수 지정

```bash
export MLFLOW_TRACKING_URI="http://127.0.0.1:5000"
```

<br>

- Experiments를 생성한 후, Run

```bash
mlflow experiments create --experiment-name my-experiment
```

```bash
mlflow run svc_with_hyperparams_tuning --experiment-name my-experiment --no-conda
```

<br>

- mlflow ui 화면에서 저장된 데이터를 확인할 수 있음

<br>

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
<img src='https://drive.google.com/uc?id=1zVUJLNCBlAmy3vtLSMbCtWzM-z4_4c-C' width=800/>

<br>

- `artifacts` 폴더와 `mlflow.db` 파일이 저장된 것 확인

<br>

### 4.3.3 MLflow 실제 활용 사례

MLflow Tracking Server는 하나로 통합 운영

- Tracking Server를 하나 배포하고, 팀 내 모든 Researcher가 이 Tracking Server에 실험 기록
  - Tracking Server를 배포할 때는 Docker Image, Kubernetes 등에 진행
    - 회사의 인프라에 따라 다름
- 로그나 모델이 한 곳에 저장되므로, 팀 내 모든 실험을 공유할 수 있음
- Artifact Storage와 DB 역시 하나로 운영
  - Artifact Storage는 GCS나 S3 같은 스토리지 이용
  - DB는 CloudSQL이나 Aurora RDS 같은 DB 이용
- 이 두 저장소는 Tracking Server에 의해 관리

<br>

## 4.4 Special Mission

1. 개인의 Local에 Mlflow 환경 설정 (pip install)
2. 개인의 Local에 MLflow 환경 설정 (Docker)
3. 팀에서 공통적으로 사용할 MLflow Tracking Server GCP에 배포하기